In [276]:
import os
import cv2
import numpy as np
import pytesseract
from PIL import Image

def find_pricetag_file(image_path):
    return find_pricetag_image(cv2.imread(image_path))


def find_pricetag_image(image):
    cv2.imwrite( "latest_raw.jpg", image)

    img = _get_crop(image)

    cv2.imwrite( "latest_crop.jpg", img)

    return img


def _auto_canny_otsu(image):
    otsu_thresh_val = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    high_thresh_val = otsu_thresh_val[0]
    lower_thresh_val = otsu_thresh_val[0] * 0.5

    edged = cv2.Canny(image, lower_thresh_val, high_thresh_val)

    cv2.imwrite("steps_thres.jpg", otsu_thresh_val[1])
    cv2.imwrite("steps_edged.jpg", edged)

    return edged


def _get_pricetag_contour(mat):
    bw = cv2.cvtColor(mat, cv2.COLOR_BGR2GRAY)
    
    kernel = np.ones((5,5),np.uint8)
    bw = cv2.dilate(bw,kernel,iterations = 4)
    
    edged = _auto_canny_otsu(bw)
    
    img, contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:1]

    return contours


def _get_crop(original_image):
    scale_mult = 0.15
    mat = cv2.resize(original_image, (0, 0), fx=scale_mult, fy=scale_mult)
    contour = _get_pricetag_contour(mat)

    # Obtain shape approximation
    card = contour[0]
    peri = cv2.arcLength(card, True)
    approx = cv2.approxPolyDP(card, 0.02 * peri, True)

    x, y, w, h = cv2.boundingRect(card)
    w /= scale_mult
    h /= scale_mult

    w = int(w)
    h = int(h)

    # Test to see if our approximation is a quadrilateral
    if len(approx) != 4:
        rect = rectify(cv2.boxPoints(cv2.minAreaRect(card)))
    else:
        # Great! We can safely apply our perspective correction technique
        rect = rectify(approx)

    for i in range(4):
        # Compute all 4 corners on original image (re-apply scaling)
        rect[i][0] /= scale_mult
        rect[i][1] /= scale_mult
        
    cv2.imwrite("steps_contours_bon.jpg", original_image)

    # Output perspective correction to an image of about the same size
    output = np.array([[0, 0], [w-1, 0], [w-1, h-1], [0, h-1]], np.float32)
    transform = cv2.getPerspectiveTransform(rect, output)
    return cv2.warpPerspective(original_image, transform, (w, h))


def rectify(h):
    h = h.reshape((4, 2))
    hnew = np.zeros((4, 2), dtype=np.float32)

    add = h.sum(1)
    hnew[0] = h[np.argmin(add)]
    hnew[2] = h[np.argmax(add)]

    diff = np.diff(h, axis=1)
    hnew[1] = h[np.argmin(diff)]
    hnew[3] = h[np.argmax(diff)]

    return hnew



<img src="WuUk3Cu3-ps.jpg", width=700, height=400>

In [277]:
img_path = "WuUk3Cu3-ps.jpg"
img = find_pricetag_file(img_path)

cropped = cv2.imread("latest_crop.jpg", 0)
retval, cropped = cv2.threshold(cropped, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

kernel = np.ones((2,2),np.uint8)
cropped = cv2.erode(cropped,kernel,iterations = 1)
cv2.imwrite( "latest_crop.jpg", cropped)

True

<img src="screen/1.png", width=500, height=400>

In [278]:
cv2.imshow('img',cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [279]:
pytesseract.pytesseract.tesseract_cmd = 'D:/Soft/Tesseract-OCR/tesseract.exe'
tessdata_dir_config = '--tessdata-dir "D:/Soft/Tesseract-OCR/tessdata"'
print(pytesseract.image_to_string(Image.open("latest_crop.jpg"), lang='rus', config = tessdata_dir_config))

Пиво БОЯРИНЪ ЖИВОЕ 1.4211

свиное 4,5% №1

эти тихсос. россия :
109
|

\\\\\\П\№\\|\\\№\Ш


<img src="1GPp4A4zA1E.jpg", width = 700, height = 400>

In [280]:
img_path = "1GPp4A4zA1E.jpg"
img = find_pricetag_file(img_path)

cropped = cv2.imread("latest_crop.jpg", 0)
retval, cropped = cv2.threshold(cropped, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

kernel = np.ones((2,2),np.uint8)
cropped = cv2.dilate(cropped,kernel,iterations = 1)
cv2.imwrite( "latest_crop.jpg", cropped)

True

<img src="screen/2.png", width=500, height=400>

In [281]:
cv2.imshow('img',cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [282]:
pytesseract.pytesseract.tesseract_cmd = 'D:/Soft/Tesseract-OCR/tesseract.exe'
tessdata_dir_config = '--tessdata-dir "D:/Soft/Tesseract-OCR/tessdata"'
print(pytesseract.image_to_string(Image.open("latest_crop.jpg"), lang='rus', config = tessdata_dir_config))

Пиво БОЯРИНЪ ЖИВОЕ 1.42л

светлое 4,5% ПЭТ
\
здвод трнхсос. россия ‘

109”

… НШНПШН НПНШППНН № ”‘*- *$


<img src="1UGHGHRQKxU.jpg", width = 700, height = 400>

In [284]:
img_path = "1UGHGHRQKxU.jpg"
img = find_pricetag_file(img_path)

cropped = cv2.imread("latest_crop.jpg", 0)
retval, cropped = cv2.threshold(cropped, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

kernel = np.ones((5,5),np.uint8)
cropped = cv2.dilate(cropped,kernel,iterations = 1)
cropped = cv2.erode(cropped,kernel,iterations = 1)

cv2.imwrite( "latest_crop.jpg", cropped)

True

<img src="screen/3.png", width=500, height=400>

In [285]:
cv2.imshow('img',cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [286]:
pytesseract.pytesseract.tesseract_cmd = 'D:/Soft/Tesseract-OCR/tesseract.exe'
tessdata_dir_config = '--tessdata-dir "D:/Soft/Tesseract-OCR/tessdata"'
print(pytesseract.image_to_string(Image.open("latest_crop.jpg"), lang='rus', config = tessdata_dir_config))

„..:, % "…“? „…

ЗАВОД '.:‘і'. ' СЕ": |". "'СИЧ

109,”

| ”| П .' .'”?!5 Ш!" ||!


<img src="96mXYYJnomk.jpg", width = 700, height = 400>

In [287]:
img_path = "96mXYYJnomk.jpg"
img = find_pricetag_file(img_path)

cropped = cv2.imread("latest_crop.jpg", 0)
retval, cropped = cv2.threshold(cropped, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

cv2.imwrite( "latest_crop.jpg", cropped)

True

<img src="screen/4.png", width=500, height=400>

In [288]:
cv2.imshow('img',cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [289]:
pytesseract.pytesseract.tesseract_cmd = 'D:/Soft/Tesseract-OCR/tesseract.exe'
tessdata_dir_config = '--tessdata-dir "D:/Soft/Tesseract-OCR/tessdata"'
print(pytesseract.image_to_string(Image.open("latest_crop.jpg"), lang='rus', config = tessdata_dir_config))

Пиво БОЯРИНЪ ЖИВОЕ !.

светлое 4,5% ПЭТ
ЗАВОД ЁРЕХСОС. РОССИЯ

109

|” НП||Н| | ППШППШШШШШ __Ё'Ё:


<img src="IMG_3223.jpg", width = 400, height = 300>

In [290]:
img_path = "IMG_3223.jpg"
img = find_pricetag_file(img_path)

cropped = cv2.imread("latest_crop.jpg", 0)
retval, cropped = cv2.threshold(cropped, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

kernel = np.ones((5,5),np.uint8)
cropped = cv2.dilate(cropped,kernel,iterations = 1)

cv2.imwrite( "latest_crop.jpg", cropped)

True

<img src="screen/5.png", width=500, height=400>

In [291]:
cv2.imshow('img',cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [292]:
pytesseract.pytesseract.tesseract_cmd = 'D:/Soft/Tesseract-OCR/tesseract.exe'
tessdata_dir_config = '--tessdata-dir "D:/Soft/Tesseract-OCR/tessdata"'
print(pytesseract.image_to_string(Image.open("latest_crop.jpg"), lang='rus', config = tessdata_dir_config))

Напиток ПЕПСИ-КОЛА 1.75л

безалкогольный газир.ПЭТ

Рерзісо Но|с|іп

 

пншнншилшлшппишиш №№


<img src="IMG_3254.jpg", width = 400, height = 300>

In [293]:
img_path = "IMG_3254.jpg"
img = find_pricetag_file(img_path)

cropped = cv2.imread("latest_crop.jpg", 0)
retval, cropped = cv2.threshold(cropped, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

kernel = np.ones((5,5),np.uint8)
cropped = cv2.dilate(cropped,kernel,iterations = 1)

cv2.imwrite( "latest_crop.jpg", cropped)

True

<img src="screen/6.png", width=500, height=400>

In [294]:
cv2.imshow('img',cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [295]:
pytesseract.pytesseract.tesseract_cmd = 'D:/Soft/Tesseract-OCR/tesseract.exe'
tessdata_dir_config = '--tessdata-dir "D:/Soft/Tesseract-OCR/tessdata"'
print(pytesseract.image_to_string(Image.open("latest_crop.jpg"), lang='rus', config = tessdata_dir_config))

. БОЯРИНЪ ЖИВОЕ 1.4211

06 4,5% ПЭТ
ЁРЕХСОС. РОССИЯ

109°°

|№”…НННПППНН !!!!! №№


<img src="IMG_20180506_214436.jpg", width = 400, height = 300>

In [296]:
img_path = "IMG_20180506_214436.jpg"
img = find_pricetag_file(img_path)

cropped = cv2.imread("latest_crop.jpg", 0)
retval, cropped = cv2.threshold(cropped, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

kernel = np.ones((5,5),np.uint8)
cropped = cv2.dilate(cropped,kernel,iterations = 2)

cv2.imwrite( "latest_crop.jpg", cropped)

True

<img src="screen/7.png", width=500, height=400>

In [297]:
cv2.imshow('img',cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [298]:
pytesseract.pytesseract.tesseract_cmd = 'D:/Soft/Tesseract-OCR/tesseract.exe'
tessdata_dir_config = '--tessdata-dir "D:/Soft/Tesseract-OCR/tessdata"'
print(pytesseract.image_to_string(Image.open("latest_crop.jpg"), lang='rus', config = tessdata_dir_config))

Напиток ПЕПСИ-КОЛА 1. 75л

безалкогольный газир.ПЭТ

Рерзісо Ноісііп

95°°

і////!/!|!/Н/…!ЁЁ!!!Е ”"' !ННМНН Ц"*`**‹-Ётд…х…ітт:‘%


<img src="NsjCnKK_ev4.jpg", width = 400, height = 300>

In [299]:
img_path = "NsjCnKK_ev4.jpg"
img = find_pricetag_file(img_path)

cropped = cv2.imread("latest_crop.jpg", 0)
retval, cropped = cv2.threshold(cropped, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

kernel = np.ones((2,2),np.uint8)
cropped = cv2.erode(cropped,kernel,iterations = 1)
cropped = cv2.dilate(cropped,kernel,iterations = 1)


cv2.imwrite( "latest_crop.jpg", cropped)

True

<img src="screen/8.png", width=500, height=400>

In [300]:
cv2.imshow('img',cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [301]:
pytesseract.pytesseract.tesseract_cmd = 'D:/Soft/Tesseract-OCR/tesseract.exe'
tessdata_dir_config = '--tessdata-dir "D:/Soft/Tesseract-OCR/tessdata"'
print(pytesseract.image_to_string(Image.open("latest_crop.jpg"), lang='rus', config = tessdata_dir_config))

Пиво БОЯРИНЪ ЖИВОЕ 1.42п ‘

тише 4,5% пэт
ЗАВОД тпхсос. россия

10934

Ш ”№№!!!” ПШШШ||||||Е||| “№…;


<img src="OQ0sPppXGRY.jpg", width = 400, height = 300>

In [302]:
img_path = "OQ0sPppXGRY.jpg"
img = find_pricetag_file(img_path)

cropped = cv2.imread("latest_crop.jpg", 0)
retval, cropped = cv2.threshold(cropped, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

kernel = np.ones((2,2),np.uint8)
cropped = cv2.erode(cropped,kernel,iterations = 1)

cv2.imwrite( "latest_crop.jpg", cropped)

True

<img src="screen/9.png", width=500, height=400>

In [303]:
cv2.imshow('img',cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [304]:
pytesseract.pytesseract.tesseract_cmd = 'D:/Soft/Tesseract-OCR/tesseract.exe'
tessdata_dir_config = '--tessdata-dir "D:/Soft/Tesseract-OCR/tessdata"'
print(pytesseract.image_to_string(Image.open("latest_crop.jpg"), lang='rus', config = tessdata_dir_config))

Пиво БОЯРИНЪ ЖИВОЕ '|.-

светлое 4,5% пэт
ЗАВОД ':‘РЕХСОС. россия

109

|||ншн|ипнпшдш№ш5


<img src="tUb4iYk_cYk.jpg", width = 400, height = 300>

In [305]:
img_path = "tUb4iYk_cYk.jpg"
img = find_pricetag_file(img_path)

cropped = cv2.imread("latest_crop.jpg", 0)
retval, cropped = cv2.threshold(cropped, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

cv2.imwrite( "latest_crop.jpg", cropped)

True

<img src="screen/10.png", width=500, height=400>

In [306]:
cv2.imshow('img',cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [307]:
pytesseract.pytesseract.tesseract_cmd = 'D:/Soft/Tesseract-OCR/tesseract.exe'
tessdata_dir_config = '--tessdata-dir "D:/Soft/Tesseract-OCR/tessdata"'
print(pytesseract.image_to_string(Image.open("latest_crop.jpg"), lang='rus', config = tessdata_dir_config))

Пиво БОЯРИНЪ ЖИВОЕ 1.42п

светлое 04, 5% ПЭТ
ЗАВОД Т.РЕХСОС РОССИЯ

109.”

\

Ш““ИПП”!”|№НШ|||||||||||| “&=—№№…„щц


<img src="WuUk3Cu3-ps.jpg", width = 400, height = 300>

In [308]:
img_path = "WuUk3Cu3-ps.jpg"
img = find_pricetag_file(img_path)

cropped = cv2.imread("latest_crop.jpg", 0)
retval, cropped = cv2.threshold(cropped, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

kernel = np.ones((2,2),np.uint8)
cropped = cv2.erode(cropped,kernel,iterations = 1)

cv2.imwrite( "latest_crop.jpg", cropped)

True

<img src="screen/11.png", width=500, height=400>

In [309]:
cv2.imshow('img',cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [310]:
pytesseract.pytesseract.tesseract_cmd = 'D:/Soft/Tesseract-OCR/tesseract.exe'
tessdata_dir_config = '--tessdata-dir "D:/Soft/Tesseract-OCR/tessdata"'
print(pytesseract.image_to_string(Image.open("latest_crop.jpg"), lang='rus', config = tessdata_dir_config))

Пиво БОЯРИНЪ ЖИВОЕ 1.4211

свиное 4,5% №1

эти тихсос. россия :
109
|

\\\\\\П\№\\|\\\№\Ш
